In [ ]:
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "packages")

In [ ]:
import os

if os.environ.get("CIRCLECI"):
    default_env = os.environ.get("CONDA_DEFAULT_ENV")
    os.environ["PYSPARK_DRIVER_PYTHON"] = (
        f"/home/circleci/miniconda/envs/{default_env}/bin/python"
    )
    os.environ["PYSPARK_PYTHON"] = (
        f"/home/circleci/miniconda/envs/{default_env}/bin/python"
    )

# Time Series Experiment: Tall vs Wide


## Purpose

The aim of this experiment was to investigate the difference in the processing time for
calculating window features on tall vs wide format on large volume of data. With the
underlying assumption that in wide format, we need to collect the data into an array
once then calculate many times, as opposed to tall format where each window function
will re-order the data each time (depending on the spark optimizer).


## Hypothesis

We believe that leveraging arrays will result in faster run time because we simply
need to collect the array once and then calculate many times. As opposed to tall format
where we may need to shuffle and re-order the data multiple times (spark optimizer dependent).

We want to find out two things:

* Hypothesis A - The run time performance for the same number of rows using arrays
  is equivalent to using native window functions in tall format.
* Hypothesis B - The run time performance using arrays can be faster when we only
  need certain rows. For example, we only need the window calculation value as at the end-of-month.
  In tall format, this would require windowing all 30 days therefore 30 rows. But in
  wide format, we only perform the calculation on 1 row.



## General description
### Tall Format

When you think 'tall' think 'vertical'. Here, every row represents an
observation belonging to a particular category. Note that in spark, row ordering
is not preserved.
```
| hcp_id | time_index | value |
|--------|------------|-------|
| h1     | 1          | 10    |
| h1     | 2          | 3     |
| h1     | 3          | 4     |
| h1     | 4          | 2     |
| h1     | 5          | 7     |
| h1     | 6          | 2     |
| h2     | 1          | 4     |
| h2     | 2          | 7     |
| h2     | 3          | 2     |
| h2     | 4          | 8     |
| h2     | 5          | 2     |
| h2     | 6          | 9     |
```
### Wide Format
When you think 'wide', think 'horizontal’. Here, categorical data is always grouped.
You can think of it as a summary of long data. The order is always preserved once collected.

```
| hcp_id | time_index_array | value_array    |
|--------|------------------|----------------|
| h1     | [1,2,3,4,5,6]    | [10,3,4,2,7,2] |
| h2     | [1,2,3,4,5,6]    | [4,7,2,8,2,9]  |
```

## Data Processing Logic

For both formats, the pre-requisite is to join the datasets with spine
(dataset at unit of analysis grain) in order to have an equivalent comparison
(Hypothesis A).

```
Spine:
| hcp_id | time_index |
|--------|------------|
| h1     | 1          |
| h1     | 2          |
| h1     | 3          |
| h1     | 4          |
| h1     | 5          |
| h1     | 6          |
| h2     | 1          |
| h2     | 2          |
| h2     | 3          |
| h2     | 4          |
| h2     | 5          |
| h2     | 6          |
```

Let's assume the input data is at day level and we want to calculate features at the month level.
The order of operation in both formats would be:

- Tall:
  - Partition by key(s) and aggregate over the given window range at the daily level.
  - Filter to keep only month-end rows.

![Tall_processing](images/tall_processing.png)

- Wide:
  - Filter input dataset to keep only month-end rows.
  - An anchor (time_index from spine) is used to determine the current row. We find
    out the anchor position in time_index_array, relative to which the window range
    is determined.
  - Based on the window range indices, the elements in value_array are aggregated.

![Wide_processing](images/wide_processing.png)

Note the ordering of the filter statement. In the wide format, we can apply the filter
first which will massively reduce the number of rows (Hypothesis B).

## Cluster Configuration

For this experiment, we used a Databricks cluster with following config:

```
| Memory       | 256gb                  |
|--------------|------------------------|
| Cores        | 32                     |
| Worker nodes | Scalable till 40 nodes |
```

Adjustments done for this experiment:

- Spark version upgraded to 3.1.2 (9.1 LTS Databricks runtime). Slicing doesn't
  work properly on spark 3.0.1 or less.
- Having a lot of array columns often fill up the containers and causes executors
  to go out of memory. Selecting only the required columns while writing the
  dataframes helped manage the issue and decreased the execution time too.
- Faced intermittent "AWS insufficient Instance Capacity failure" issues, when
  we had all nodes as spot instances and AWS wasn't able to find any instances.
  As a result the cluster would not spin up. To resolve this we adjusted the cluster
  config to have 2 on-demand instances and 38 spot instances.

We performed the same calculations twice, once with X amount of data (Experiment 1),
and then again with 2X amount of data (Experiment 2).

## Experiment 1

### Input data

Below are the details of the input data for tall and wide
aggregation over window steps.

```
|           | Tall        | Wide        |
|-----------|-------------|-------------|
| Row count | 4.7 Billion | 4.7 Billion |
```


### Preprocessing Times

```
| Format   | Type of operation      | Time   |
|----------|------------------------|--------|
| Tall     | Join with spine        | 10 min |
| Wide     | Join with spine        | 58 min |
| Wide     | Collect list of values | 2 min  |
| Wide eom | Filter for EOM         | 6 min  |
```

EOM: For situations where we need to calculate features over windows for end of the month,
  the processing logic changes a bit.

Tall EOM: Filter is applied after features for tall are generated.

Wide EOM: Filter is applied before the feature calculation since we have all the
  information for lookback/forward in array columns.


### Processing Time

```
| Count of features    | Tall    | Wide      | Wide EOM |
|----------------------|---------|-----------|----------|
| 3 features 1 window  | 5.6 min | 11 min    | 41 sec   |
| 3 features 2 windows | 4 min   | 12.3 min  | 1 min    |
| 3 features 3 windows | 7.7 min | 14 min    | 1.1 min  |
| 3 features 5 windows | 9.5 min | 13.75 min | 2.6  min |
```


### Postprocessing Time

```
| Format | Type of operation | Time  |
|--------|-------------------|-------|
| Tall   | Filter for EOM    | 2 min |
```


## Experiment 2

### Input Data

Below are the details of the input data for tall and wide
aggregation over window steps.

```
|           | Tall        | Wide        |
|-----------|-------------|-------------|
| Row count | 9.4 Billion | 9.4 Billion |
```


### Preprocessing Time

```
| Format   | Type of operation      | Time    |
|----------|------------------------|---------|
| Tall     | Join with spine        | 33 min  |
| Wide     | Join with spine        | 156 min |
| Wide     | Collect list of values | 41 min  |
| Wide eom | Filter for EOM         | 15 min  |
```


### Processing Time

```
| Count of features    | Tall     | Wide     | Wide EOM |
|----------------------|----------|----------|----------|
| 3 features 1 window  | 12.3 min | 25.6 min | 2.1 min  |
| 3 features 2 windows | 23 min   | 25.2 min | 2 min    |
| 3 features 3 windows | 40 min   | 24 min   | 2.1 min  |
| 3 features 5 windows | 58.6 min | 25 min   | 2.4  min |
```


### Postprocessing Time

```
| Format | Type of operation | Time  |
|--------|-------------------|-------|
| Tall   | Filter for EOM    | 1 min |
```


## Observations

1. In comparison, tall is more optimised when processing same
  number of records but for cases where we need to calculate features for EOM/EOW etc.
  Wide ranges from 5x ~ 10x faster.

2. Considering the pre-processing time, tall process is faster but
  processing time for wide becomes favourable as we increase the number of records and
  windows. It is unclear why in Experiment 2 the run time does not increase linearly.

3. Databricks has some instability under the hood, resulting in non-linear
  run times in certain cases.

4. The cost of generating the array structure can be considered low. Though
  it is not clear how or why the run time would be from 2 minutes to 40 minutes. Though
  anecdotally we've seen that `collect_list` is performant in most circumstances
  beyond this experiment.

5. The cost of joining the array structure to the spine is expensive. This makes
  sense as the data is being replicated many times.

## Conclusion

### Hypothesis A

For the same number of rows, the wide implementation cannot be said to be worse or better than
tall native window functions. In Experiment 1, the implementation was a few minutes slower,
but in Experiment 2, the implementation was ~20 minutes faster in some cases. However,
the process to generate the array for every row in the spine would make this process
overall slower.

### Hypothesis B

When we only need the window calculation for specific rows, the wide implementation is
an order of magnitude faster. In this situation, the additional cost would be the
collect process and the filter, the inclusion of which on larger datasets may potentially
be worth it.


## Recommendations

When it comes to performing window calculations for every row, the tall format is generally
still recommended. The wide format would require duplicating the data multiple times and
in big data settings, this is not recommended.

When it comes to needing the window calculation for only specific rows, the wide format
is faster one can filter the data down the only the specific rows then perform the calculation.

# Appendix
## Data Processing Steps

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.ui.showConsoleProgress", False).getOrCreate()
import datetime

import pyspark.sql.functions as f

from pyspark.sql.types import (
    IntegerType,
    StringType,
    StructField,
    StructType,
    TimestampType,
)

from feature_generation.v1.core.timeseries.array_collect import (
    collect_array_then_interpolate,
)


schema = StructType(
    [
        StructField("element_id", StringType(), True),
        StructField("reading_ts", TimestampType(), True),
        StructField("reading_val", IntegerType(), True),
    ]
)

data = [
    ("line1", datetime.datetime(1970, 1, 1, 5, 0), 1),
    ("line1", datetime.datetime(1970, 1, 1, 5, 2), 2),
    ("line1", datetime.datetime(1970, 1, 1, 5, 6), 3),
    ("line1", datetime.datetime(1970, 1, 1, 5, 7), 4),
    ("line1", datetime.datetime(1970, 1, 1, 5, 12), 5),
    ("line2", datetime.datetime(1970, 1, 1, 5, 0), 3),
    ("line2", datetime.datetime(1970, 1, 1, 5, 5), 1),
    (
        "line2",
        datetime.datetime(1970, 1, 1, 5, 10),
        8,
    ),
    (
        "line2",
        datetime.datetime(1970, 1, 1, 5, 11),
        6,
    ),
    (
        "line2",
        datetime.datetime(1970, 1, 1, 5, 12),
        7,
    ),
]

mock_datetime_df = spark.createDataFrame(data, schema)

Input data:

In [ ]:
from feature_generation.v1.core.timeseries.datetime import minute_index

mock_datetime_df = mock_datetime_df.withColumn(
    "minute_index", minute_index("reading_ts")
)
mock_datetime_df.show(truncate=False)

Spine:

In [ ]:
schema = StructType(
    [
        StructField("element_id", StringType(), True),
        StructField("reading_ts", TimestampType(), True),
    ]
)

data = [
    ("line1", datetime.datetime(1970, 1, 1, 5, 0)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 1)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 2)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 3)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 4)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 5)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 6)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 7)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 8)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 9)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 10)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 11)),
    ("line1", datetime.datetime(1970, 1, 1, 5, 12)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 0)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 1)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 2)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 3)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 4)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 5)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 6)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 7)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 8)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 9)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 10)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 11)),
    ("line2", datetime.datetime(1970, 1, 1, 5, 12)),
]

spine_df = spark.createDataFrame(data, schema)

spine_df = spine_df.withColumn("minute_index", minute_index("reading_ts"))
spine_df.show(truncate=False)

### Preprocessing step

Tall:

1. Join with spine

In [ ]:
tall_df = spine_df.join(
    mock_datetime_df, ["element_id", "reading_ts", "minute_index"], "left"
)

In [ ]:
tall_df = tall_df.orderBy("element_id", "reading_ts")

In [ ]:
tall_df.show(truncate=False)

Wide:
1. Collect list of values

In [ ]:
from feature_generation.v1.core.timeseries.array_collect import (
    collect_array_then_interpolate,
)
from feature_generation.v1.core.timeseries.array_transform import interpolate_constant

collected_wide_df = collect_array_then_interpolate(
    df=mock_datetime_df,
    order="minute_index",
    values=["reading_val"],
    groupby="element_id",
    interpolate_func=interpolate_constant,
    constant=0,
).drop("minute_index")

collected_wide_df.show(truncate=False)

2. Join with spine

In [ ]:
wide_df = spine_df.join(collected_wide_df, ["element_id"], "left")

wide_df.show(truncate=False)

### Processing step

Tall:

In [ ]:
from feature_generation.v1.core.features.create_column import create_columns_from_config
from feature_generation.v1.core.features.windows import generate_window_grid
import pyspark.sql.functions as f

tall_grid_dict = {
    "inputs": ["reading_val"],
    "funcs": [f.sum],
    "windows": [
        {
            "partition_by": ["element_id"],
            "order_by": ["minute_index"],
            "descending": False,
        },
    ],
    "ranges_between": [
        [-6, -1],
        [-3, -1],
    ],
    "negative_term": "past",
    "positive_term": "next",
    "prefix": "ftr_",
}


df_tall_ft = create_columns_from_config(
    df=tall_df,
    column_instructions=[
        generate_window_grid(
            **tall_grid_dict,
        )
    ],
)


df_tall_ft.show(truncate=False)

Wide:

In [ ]:
from feature_generation.v1.core.features.windows import (
    aggregate_over_slice_grid,
)
from feature_generation.v1.core.timeseries.array_aggregate import array_sum

wide_grid_dict = {
    "inputs": ["reading_val_array_padded"],
    "funcs": [array_sum],
    "anchor_col": "minute_index",
    "anchor_array": "spine_index",
    "ranges_between": [
        [-6, -1],
        [-3, -1],
    ],
    "prefix": "ftr_",
}

df_wide_ft = create_columns_from_config(
    df=wide_df,
    column_instructions=[
        aggregate_over_slice_grid(
            **wide_grid_dict,
        )
    ],
)

df_wide_ft.select(
    "element_id",
    "reading_ts",
    "minute_index",
    "ftr_reading_val_array_padded_array_sum_past_6_past_1",
    "ftr_reading_val_array_padded_array_sum_past_3_past_1",
).show(truncate=False)